In [28]:
insert_batch_size = 1000
query_count = 20


dataverse = "LocationDb"
table_name = "Locations_RTree_Constant"

dbHost = "http://localhost:19002"
benchmark_plan_path = f"commands/benchmark_plan_{dataverse}_{table_name}_{insert_batch_size}_{query_count}.sql"

# None means no limit
debug_max_batches = 10

In [29]:
data =[]
with open("data/point_data_centroid.csv", "r") as f:
    f.readline()
    for line in f:
        row = line.strip().split(",")
        id = row[0]
        locationX = row[1]
        locationY = row[2]
        description = row[3]
        data.append([id, locationX, locationY, description])

In [30]:
import random
import requests
import math

def point(x, y):
    return f'point("{x},{y}")'

def rectangle(x1, y1, x2, y2):
    return f'rectangle("{x1},{y1} {x2},{y2}")'

def generate_insert_command(batch):
    query_lines = []
    query_lines.append(f"INSERT INTO {table_name} ([")
    for row in batch:
        id = row[0]
        location_x = row[1]
        location_y = row[2]
        description = row[3]
        is_last = (row == batch[-1])
        query_lines.append(f'{{"id": {id}, "location": {point(location_x, location_y)}, "description": "{description}"}}{ "" if is_last else "," }')
    query_lines.append("])")
    
    return "".join(query_lines)

def generate_query_commands(inserted_points):
    queries = []
    selected_points = random.sample(inserted_points, query_count)
    for selected_point in selected_points:
        location_x = selected_point[0]
        location_y = selected_point[1]
        stdev_x = 60
        stdev_y = 20
        stdev_scale = .03
        offset_x = random.random() * stdev_x * stdev_scale
        offset_y = random.random() * stdev_y * stdev_scale
        start_x = float(location_x) - offset_x
        start_y = float(location_y) - offset_y
        end_x = float(location_x) + offset_x
        end_y = float(location_y) + offset_y
        
        query = f"SELECT location, description FROM {table_name} WHERE spatial_intersect(location, {rectangle(start_x, start_y, end_x, end_y)});";
        queries.append(query)
    return queries

In [31]:
inserted_points = []
current_batch = 0
commands = []
while True:
    if debug_max_batches is not None and current_batch >= debug_max_batches:
        break
    batch_to_insert = data[current_batch * insert_batch_size:(current_batch + 1) * insert_batch_size]
    if not batch_to_insert: # no more data to insert
        break
    
    insert_command = generate_insert_command(batch_to_insert)
    inserted_points.extend(map(lambda x: (x[1], x[2]), batch_to_insert))
    
    query_commands = generate_query_commands(inserted_points)
    commands.append(insert_command)
    commands.extend(query_commands)
    
    current_batch += 1
   
import os 
# write the commands to a file
benchmark_dir = os.path.dirname(benchmark_plan_path)
if not os.path.exists(benchmark_dir):
    os.makedirs(benchmark_dir)
with open(benchmark_plan_path, "w") as f:
    for command in commands:
        f.write(command + "\n")
    

In [32]:
import logging
import datetime
import os

logs_dir = "./logs"
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir, exist_ok=True)


In [49]:
import time
import json

def execute_command(command, client_context_id=None):
    return requests.post(f"{dbHost}/query/service", data={
        'statement': command,
        'dataverse': dataverse,
        'client_context_id': client_context_id,
    })

log_file_path = f"./logs/benchmark_{dataverse}_{table_name}_{insert_batch_size}_{query_count}_{str(time.time())}.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename = log_file_path,
    filemode = 'a'
)
print(f"Logging to file")
print(f"To see the logs in real time: tail -f {log_file_path}")
logger = logging.getLogger()

with open(log_file_path, "a") as log:
    with open(benchmark_plan_path, "r") as f:
        index = 0
        for line in f:
            index += 1
            command = line.strip()
            if command:
                trace_id = str(index).zfill(10)
                log.write(json.dumps({"trace-id": trace_id,"message": "command.execute",}) + "\n")
                response = execute_command(command, client_context_id=trace_id)
                if response.status_code != 200:
                    log.write(json.dumps({
                        "trace_id": json_response.get("clientContextID"),
                        "message": "command.execute",
                        "http.status": response.status_code,
                        "status": json_response.get("status"),
                        "metrics": json_response.get("metrics"),
                    }) + "\n")
                else:
                    json_response = response.json()
                    print(json_response)
                    log.write(json.dumps({
                        "trace_id": json_response.get("clientContextID"),
                        "message": "command.execute",
                        "http.status": response.status_code,
                        "status": json_response.get("status"),
                        "result_count": len(json_response.get("results", [])) if json_response.get("results") else None,
                        "metrics": json_response.get("metrics"),
                        
                    }) + "\n")
                        


Logging to file
To see the logs in real time: tail -f ./logs/benchmark_LocationDb_Locations_RTree_Constant_1000_20_1745688865.1779342.log
{'requestID': '0554868d-1e92-4e98-a890-d452e7ecbf9c', 'clientContextID': '0000000002', 'signature': {'*': '*'}, 'results': [{'location': [28.840504824704134, 55.22252909089843], 'description': '39023e9c7e'}, {'location': [28.777382628658415, 54.944082629219686], 'description': 'f546b3ee37'}, {'location': [28.75225238343412, 55.241870701658286], 'description': '31fa8e5be1'}], 'plans': {}, 'status': 'success', 'metrics': {'elapsedTime': '18.74325ms', 'executionTime': '18.527291ms', 'compileTime': '4.334083ms', 'queueWaitTime': '0ns', 'resultCount': 3, 'resultSize': 256, 'processedObjects': 6}}
{'requestID': '3241e8cc-2163-4eca-a21a-c989888bc58e', 'clientContextID': '0000000003', 'signature': {'*': '*'}, 'results': [{'location': [9.242693791935386, 39.869942564765424], 'description': 'c91c76f528'}, {'location': [9.497262760944068, 39.824250971945276], '